In [ ]:
!pip install transformers gradio tensorflow emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 42.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving archive (2).zip to archive (2).zip
Saving ReactionGIF.ids.json to ReactionGIF.ids.json
Saving face_emotion.h5 to face_emotion.h5
Saving emotiontxt.txt to emotiontxt.txt


In [ ]:
import os
print("face_emotion.h5" in os.listdir())

True


In [ ]:
from transformers import pipeline
import tensorflow as tf
from tensorflow.keras.models import load_model
import gradio as gr
import emoji, json, re, numpy as np
from PIL import Image

# Load pretrained DistilBERT for text sentiment analysis:contentReference[oaicite:5]{index=5}
text_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
!pip install keras opencv-python

In [ ]:
# Define a mapping from specific emoji characters to basic sentiments
emoji_sentiment_map = {
    "😀":"happy", "😃":"happy", "😄":"happy", "😁":"happy",
    "😆":"happy", "😅":"happy", "😂":"happy", "😊":"happy", "🙂":"happy",
    "😍":"happy", "😇":"happy",
    "😢":"sad", "😭":"sad", "😥":"sad", "😓":"sad",
    "😔":"sad", "😞":"sad", "😩":"sad", "😫":"sad", "☹️":"sad",
    "😠":"angry", "😡":"angry", "🤬":"angry", "😤":"angry",
    "😾":"angry"
}

In [ ]:
import os
import json

# Confirm the correct path
file_path = "ReactionGIF.ids.json"

if os.path.exists(file_path):
    gif_sentiment_map = {}
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            entry = json.loads(line)
            gif_sentiment_map[str(entry["original_id"])] = entry["label"]
    print("GIF sentiment map loaded successfully.")
else:
    print("File not found. Please ensure 'ReactionGIF.ids.json' is uploaded to the same directory.")

GIF sentiment map loaded successfully.


In [ ]:
def analyze(text, face_img, gif_id, emoji_img):
    # 1. Text sentiment with DistilBERT
    if text:
        res = text_analyzer(text)[0]
        sent = res['label'].capitalize()
        score = res['score'] * 100
        text_result = f"Text Sentiment: {sent} ({score:.1f}%)"
    else:
        text_result = "Text Sentiment: (no text provided)"

    # 2. Face image emotion
    if face_img:
        face_gray = face_img.convert('L').resize((48,48))
        arr = np.array(face_gray) / 255.0
        arr = arr.reshape((1,48,48,1))
        pred = face_model.predict(arr)
        label = face_labels[np.argmax(pred)]
        face_result = f"Face Emotion: {label}"
    else:
        face_result = "Face Emotion: (no image)"

    # 3. Emoji sentiment (from text)
    emojis_found = emoji.emoji_list(text) if text else []
    if emojis_found:
        sentiments = set()
        for item in emojis_found:
            char = item['emoji']
            sentiments.add(emoji_sentiment_map.get(char, "neutral"))
        emoji_sent = ", ".join(sorted(sentiments))
        emoji_result = f"Emoji Sentiment: {emoji_sent}"
    else:
        emoji_result = "Emoji Sentiment: (no emojis found)"

    # 4. GIF ID sentiment (yes → Positive, no → Negative)
    if gif_id:
        label = gif_sentiment_map.get(gif_id)
        if label:
            gif_result = f"GIF Sentiment: {'Positive' if label=='yes' else 'Negative'}"
        else:
            gif_result = "GIF Sentiment: (ID not found)"
    else:
        gif_result = "GIF Sentiment: (no ID provided)"

    return text_result, face_result, emoji_result, gif_result

In [ ]:
iface = gr.Interface(
    fn=analyze,
    inputs=[
        gr.Textbox(lines=2, label="Input Text (with emojis)"),
        gr.Image(type="pil", label="Face Image"),
        gr.Textbox(lines=1, label="Optional GIF Tweet ID"),
        gr.Image(type="pil", label="Optional Emoji Image")
    ],
    outputs=[
        gr.Textbox(label="Text Sentiment"),
        gr.Textbox(label="Face Emotion"),
        gr.Textbox(label="Emoji Sentiment"),
        gr.Textbox(label="GIF Sentiment")
    ],
    title="Multimodal Sentiment Analysis",
    description="Analyze sentiment from text, emojis, face, and GIF inputs.",
    theme=gr.themes.Monochrome()  # uses a dark background with light text
)

iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f470fb62a23d3c294b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from tensorflow.keras.models import load_model
import h5py

# Try to inspect the file
with h5py.File("face_emotion.h5", 'r') as f:
    print(list(f.keys()))

['conv2d', 'conv2d_1', 'conv2d_2', 'conv2d_3', 'dense', 'dense_1', 'dropout', 'dropout_1', 'dropout_2', 'flatten', 'max_pooling2d', 'max_pooling2d_1', 'max_pooling2d_2', 'top_level_model_weights']


In [ ]:
import h5py

with h5py.File("face_emotion.h5", 'r') as f:
    def print_structure(name):
        print(name)
    f.visit(print_structure)

conv2d
conv2d/conv2d
conv2d/conv2d/bias:0
conv2d/conv2d/kernel:0
conv2d_1
conv2d_1/conv2d_1
conv2d_1/conv2d_1/bias:0
conv2d_1/conv2d_1/kernel:0
conv2d_2
conv2d_2/conv2d_2
conv2d_2/conv2d_2/bias:0
conv2d_2/conv2d_2/kernel:0
conv2d_3
conv2d_3/conv2d_3
conv2d_3/conv2d_3/bias:0
conv2d_3/conv2d_3/kernel:0
dense
dense/dense
dense/dense/bias:0
dense/dense/kernel:0
dense_1
dense_1/dense_1
dense_1/dense_1/bias:0
dense_1/dense_1/kernel:0
dropout
dropout_1
dropout_2
flatten
max_pooling2d
max_pooling2d_1
max_pooling2d_2
top_level_model_weights


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),

    Flatten(),

    Dense(1024, activation='relu'),     # FIRST Dense layer
    Dense(7, activation='softmax')      # FINAL layer (7 classes)
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving face_emotion.h5 to face_emotion (1).h5


In [ ]:
!pip install keras opencv-python

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the model exactly as it was trained
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),

    Flatten(),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')  # FIXED: 6 classes instead of 7
])

# Load weights
model.load_weights("face_emotion.h5")

print("Model loaded successfully!")
# Define the model exactly as it was trained
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),

    Flatten(),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')  # FIXED: 6 classes instead of 7
])

# Load weights
model.load_weights("face_emotion.h5")

print("Model loaded successfully!")

Model loaded successfully!
Model loaded successfully!


In [ ]:
!pip install -q gradio transformers

In [ ]:
import gradio as gr
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Emotion labels (6 classes)
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Neutral']

# Rebuild model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(128, (3, 3), activation='relu'),

    Flatten(),
    Dense(1024, activation='relu'),
    Dense(6, activation='softmax')  # MATCHING WEIGHTS
])

# Load pretrained weights
model.load_weights("face_emotion.h5")


# Preprocessing function
def process_image(img):
    if img is None:
        return "No image uploaded."

    # Convert to grayscale
    img_gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
    img_resized = cv2.resize(img_gray, (64, 64))
    img_array = img_resized.reshape(1, 64, 64, 1).astype("float32") / 255.0

    # Predict
    prediction = model.predict(img_array)
    label = emotion_labels[np.argmax(prediction)]
    confidence = np.max(prediction)

    return f"{label} ({confidence*100:.2f}%)"


# Custom CSS for dark gold-black theme
css = """
body { background-color: black; color: white; }
.gradio-container {
    background-image: url('file/1000050514.jpg');
    background-size: cover;
    background-position: center;
    color: white;
}
"""

# Gradio interface
with gr.Blocks(css=css) as demo:
    gr.Markdown("<center><h1 style='color:gold;'>Sentiment Analysis - Face Emotion</h1></center>")

    with gr.Row():
        img = gr.Image(label="Upload Face Image", type="pil")

    out = gr.Textbox(label="Prediction Result")
    btn = gr.Button("Analyze Emotion")
    btn.click(process_image, inputs=img, outputs=out)

    gr.Markdown("<center><p style='color:white;'>Powered by CNN + Gradio UI</p></center>")

# Launch app
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c415d4cbd6f5d6b905.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
